# Load Cora Dataset 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cpu')
import numpy as np
import random

data = torch.load('data.pth')
g = data['g'].to(device)
feat = data['feat'].to(device)
label = data['label'].to(device)
train_nodes = data['train_nodes']
val_nodes = data['val_nodes']
test_nodes = data['test_nodes']
# 随机种子生成模块
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
setup_seed(20)

# Define GCN

In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    """Graph Convolution Network (GCN)

    Example
    -------
    # GCN with one hidden layer
    >>> model = GCN(100, 10, hid=32)
    """
    def __init__(self,
                 in_feats: int,
                 out_feats: int,
                 hid: list = 16,
                 dropout: float = 0.5):
        super().__init__()
        self.conv1 = GraphConv(in_feats, hid)
        self.conv2 = GraphConv(hid, out_feats)
        self.dropout = nn.Dropout(dropout)

    def forward(self, g, feat):

        if torch.is_tensor(g):
            feat = self.dropout(feat)
            feat = g @ (feat @ self.conv1.weight) + self.conv1.bias
            feat = F.relu(feat)
            feat = self.dropout(feat)
            feat = g @ (feat @ self.conv2.weight) + self.conv2.bias
            return feat
        
        g = g.add_self_loop()
        feat = self.dropout(feat)
        feat = self.conv1(g, feat)
        feat = F.relu(feat)
        feat = self.dropout(feat)
        feat = self.conv2(g, feat)
        return feat


# Train

In [ ]:
# 训练
def train():
    model.train()
    optimizer.zero_grad() # 清空梯度，防止梯度累加
    loss_fn(model(g, feat)[train_nodes], label[train_nodes]).backward()
    optimizer.step()
# 测试
@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(g, feat), []
    for nodes in (train_nodes, val_nodes, test_nodes):
        pred = logits[nodes].max(1)[1]
        acc = pred.eq(label[nodes]).float().mean()
        accs.append(acc)
    return accs


num_feats = feat.size(1)
num_classes = int(label.max() + 1)
model = GCN(num_feats, num_classes).to(device)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.

loss_fn = nn.CrossEntropyLoss()

best_val_acc = test_acc = 0
for epoch in range(1, 101):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

# Evaluate

In [ ]:
target = 1 # 确定攻击目标
target_label = label[target]
print("target label: ", target_label)

print("model predict: ", model(g, feat)[target].argmax())

In [ ]:
# save vitim model
torch.save(model.state_dict(), 'model.pth')